In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
df = pd.read_csv("dataset_001.csv")
df.head(3)

,id,text,tags
0,1,Um jovem camponês descobre um antigo artefato ...,fantasia|aventura
1,2,Dois colegas de trabalho desenvolvem sentiment...,romance|slice_of_life
2,3,"Após morrer em um acidente, um programador ren...",fantasia|isekai|aventura


In [3]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

C:\Users\rodri\PycharmProjects\Jupyter_Docker\.venv\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
embeddings = model.encode(df["text"].tolist(), normalize_embeddings=True)

In [5]:
kmeans = KMeans(n_clusters=3, random_state=42)
df["cluster"] = kmeans.fit_predict(embeddings)

In [6]:
def buscar_similares(texto, top_k=3):
    query_embedding = model.encode([texto], normalize_embeddings=True)
    scores = cosine_similarity(query_embedding, embeddings)[0]

    top_idx = np.argsort(scores)[::-1][:top_k]
    return df.iloc[top_idx][["id", "text", "tags"]], scores[top_idx]

In [7]:
resultado, scores = buscar_similares(
    "Uma aventura em um mundo mágico com heróis e monstros"
)

print(resultado)
print(scores)

    id                                               text  \
20  21  Uma história sombria sobre cultos antigos e ho...   
29  30  Um caçador de monstros percorre vilarejos amal...   
2    3  Após morrer em um acidente, um programador ren...   

                        tags  
20      terror|mistério|dark  
29    fantasia|dark|aventura  
2   fantasia|isekai|aventura  
[0.6637834  0.66359234 0.634245  ]
